In [ ]:
import numpy as np
import random

In [ ]:
# Section
a = -1
b = 2

# Additional data
epsilon = 1e-6
populationSize = 100

# Iterations 
t_max_i = 20
t_max = 10

# Propabilities
probCrossover = 0.75
probMutation = 0.5

In [ ]:
def f(x):
    return x * sin(10 * pi * x) + 1

In [ ]:
def getBitArraySize(n):
    bitArraySize = 0
    
    while n > 0:
        n >>= 1
        bitArraySize += 1
        
    return bitArraySize

In [ ]:
def bitToFloat(bitArray):
    return a + ((b - a)) / (2**bitArraySize - 1) * int(bitArray, 2)

def fitness(individual):
    return f(bitToFloat(individual))

In [ ]:
def tournamentMethod(population):
    newPopulation = []
    
    while len(newPopulation) < populationSize:
        result = random.sample(population, 2)

        firstFitnessFunction = fitness(result[0])
        secondFitnessFunction = fitness(result[1])

        if firstFitnessFunction > secondFitnessFunction:
            best = result[0]
        else:
            best = result[1]  
            
        newPopulation.append(best)
    
    return newPopulation

def mutation(population):
    newPopulation = []
    
    for individual in population:
        randomProbability = np.random.random_sample()
        
        if randomProbability <= probMutation:
            charIndexToMutate = np.random.randint(low = 0, high = bitArraySize)
            charToMutate = individual[charIndexToMutate] 

            if charToMutate == '0':
                individual = individual[:charIndexToMutate] + "1" + individual[charIndexToMutate + 1:]
            else:
                individual = individual[:charIndexToMutate] + "0" + individual[charIndexToMutate + 1:]

        newPopulation.append(individual)

    return newPopulation
        
def crossover(population, propability):    
    newPopulation = []
    
    while len(newPopulation) < populationSize:
        individualOne = population[np.random.randint(low = 0, high = populationSize - 1)]
        individualTwo = population[np.random.randint(low = 0, high = populationSize - 1)]

        probabilityForCrossover = np.random.random_sample()
        if probabilityForCrossover <= propability:
            randomIndex = np.random.randint(low = 0, high = bitArraySize)

            firstHalf_IndividualOne = individualOne[:randomIndex]
            secondHalf_IndividualOne = individualOne[randomIndex:]

            firstHalf_IndividualTwo = individualTwo[:randomIndex]
            secondHalf_IndividualTwo = individualTwo[randomIndex:]

            newPopulation.append(firstHalf_IndividualOne + secondHalf_IndividualTwo)
            newPopulation.append(firstHalf_IndividualTwo + secondHalf_IndividualOne)
        else:
            newPopulation.append(individualOne)
            newPopulation.append(individualTwo)
            
    return newPopulation

In [ ]:
initialPopulation = []
population = []

n = (int)((b - a) / epsilon)
bitArraySize = getBitArraySize(n)

for i in range(populationSize):
    initialPopulation += [np.random.randint(2, size = (bitArraySize,))]
    
for i in range(populationSize):
    population.append("".join(str(x) for x in initialPopulation[i]))

for p in np.arange(0, 1, 0.1):
    print('\n-----', p, '-----')
    best = []
    i = 0
    while i <= t_max:
        population = tournamentMethod(population)
        population = mutation(population)
        population = crossover(population, p)

        fitnesses = []
        for individual in population:
            populationFitness = fitness(individual)
            fitnesses.append(populationFitness)
        best.append(max(fitnesses))

        if i > 1:
            if best[i] - best[i - 1] < epsilon:
                t_max_i += 1
        print(i, ' - ', bitToFloat(population[i]))
        if t_max_i > t_max_i:
            break

        i += 1    

    print('\nf_max = ', bitToFloat(population[i]))